In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from src import Wing, FlightCondition, WingPool, Simulation, PostProcessing, load_folder

airfoils_data, _ = load_folder("../sample_airfoils")

wing = Wing(
    spans=[1, 0.5],
    chords=[1, 0.5, 0.4],
    offsets=[0, 0, 0],
    twist_angles=[0, 0, 0],
    dihedral_angles=[0, 0, 0],
    airfoils=["NACA4424", "NACA4424", "NACA4412"],
    surface_name="wing",
    N_panels=12,
    distribution_type="cosine",
    sweep_check=False,
)
wing.generate_mesh()  # Generate wing simulation elements
flight_condition = FlightCondition(
    V_inf=20,
    nu=1.5e-5,
    rho=1.225,
    angles_of_attack=[alpha for alpha in range(1)],
    h=0,
    ground_effect_check=False,
)
wing.setup_airfoil_data(flight_condition, airfoils_data)  # Assign airfoil data to wing

# Link wing that will be simulated with flight condition
wing_pool = WingPool(wing_list=[wing], flight_condition=flight_condition)

simulation = Simulation(
    damping_factor=0.5,
    max_iter=300,
    max_residual=1e-4,
    linear_check=False,
    show_logs=True,
    simulation_mode="latest_solution",  # Solves linear version of equations before doing non linear simulation to speed up process
)

# Run simulation
simulation_results = simulation.run(wing_pool)

[wing, wing_mirrored]
Running simulation for angles between 0 and 0
Linear simulation check: False
Found solution for angle 0
number of iterations: 15


In [3]:
post_processing = PostProcessing()
coefficients = post_processing.get_coefficients(wing_pool, simulation_results)

print(coefficients[0].global_coefficients)
print(coefficients[0].surface_coefficients)

Coefficients(forces=array([ 4.31095964e-03, -6.23416249e-19,  3.03485054e-01]), moments=array([ 0.        , -0.37971982,  0.        ]), cl_distribution={})
{'wing': Coefficients(forces=array([0.00215548, 0.00021534, 0.15174253]), moments=array([ 0.11978017, -0.18985991, -0.0023641 ]), cl_distribution=array([0.43775726, 0.36961503, 0.33633051, 0.31834189, 0.30634427,
       0.29523405, 0.28237413, 0.26742476, 0.26877524, 0.26609314,
       0.22081422, 0.09946055])), 'wing_mirrored': Coefficients(forces=array([ 0.00215548, -0.00021534,  0.15174253]), moments=array([-0.11978017, -0.18985991,  0.0023641 ]), cl_distribution=array([0.43775726, 0.36961503, 0.33633051, 0.31834189, 0.30634427,
       0.29523405, 0.28237413, 0.26742476, 0.26877524, 0.26609314,
       0.22081422, 0.09946055]))}


In [5]:
coefficients_old = post_processing.get_wing_coefficients(wing_pool, simulation_results[0].G_solution, 0, wing_pool.S_ref, wing_pool.c_ref)
coefficients_old

{'wing': {'CF': array([ 4.31095964e-03, -6.23416249e-19,  3.03485054e-01]),
  'CM': array([ 0.23956034, -0.37971982, -0.00483172]),
  'Cl_distr': {'wing': array([0.43775726, 0.36961503, 0.33633051, 0.31834189, 0.30634427,
          0.29523405, 0.28237413, 0.26742476, 0.26877524, 0.26609314,
          0.22081422, 0.09946055]),
   'wing_mirrored': array([0.43775726, 0.36961503, 0.33633051, 0.31834189, 0.30634427,
          0.29523405, 0.28237413, 0.26742476, 0.26877524, 0.26609314,
          0.22081422, 0.09946055])}}}